In [2]:
import pandas as pd
import numpy as np

## Problema 1

**Eventos:**

$A:\text{"El mensaje contiene un archivo adjunto"}$

$S:\text{"El mensaje es un SPAM"}$

$A^{c}\cap S^{c}:\text{"El mensaje no tiene adunto ni es SPAM"}$

**Probabilidades:**

$$P(A) = 0.5, \quad P(S) = 0.65, \quad P(A^{c}\cap S^{c}) = 0.15$$

Por las leyes de D'Morgan y propiedades del álgebra de probabilidades:

$$P(A^{c}\cap S^{c}) = P((A\cup S)^{c}) = 1 - P(A\cup S),$$

por lo que $P(A\cup S) = 1 - 0.15 = 0.85$, y en consecuencia:

$$P(A\cap S) = P(A) + P(S) - P(A\cup S) = 0.5 + 0.65 - 0.85 = 0.3$$

$$P(A|S) = \dfrac{P(A\cap S)}{P(S)}=\dfrac{0.3}{0.65} \approx 0.46154$$

$$P(A^{c}|S^{c}) = \dfrac{P(A^{c}\cap S^{c})}{1 - P(S)} = \dfrac{0.15}{0.35} \approx 0.42871$$




In [40]:
def prob(PA,PS,P_AS):
    """
    Args:
         PA: probabilidad de que el mensaje contenga un archivo adjunto
         PS: probabilidad de que el mensaje sea SPAM
         P_AS: probabilidad de que el mensaje no tenga adjunto y tampoco sea SPAM
    """
    if PA < 0 or PS < 0 or PA > 1 or PS > 1:
        return print('Ingrese valores consistentes')
    else:
        PAuS = 1 - P_AS
        PAnS = PA + PS - PAuS
        PA1S = PAnS/PS
        P_A1_S = P_AS/(1 - PS)
        probs = [PA1S,P_A1_S]
        return pd.DataFrame(probs,columns = ['Probabil.'], index = ['Punto 1', 'Punto 2'])

In [41]:
prob(0.5,0.65,0.15)

,Probabil.
Punto 1,0.461538
Punto 2,0.428571


## Problema 2

**Punto a**

Se considera una muestra aleatoria $(X_1,X_2,\dots,X_6)$, es decir, $X_{i}\sim_{_{iid}} N(\mu,\sigma)$, donde $\sigma = 15.7$. Si 

$$Y:=\dfrac{\sum_{i=1}^{6} X_{i} - 6\mu}{\sqrt{6}\sigma},$$ 

por el teorema del límite central, se tiene que $Y\sim N(0,1)$, por lo tanto, el problema se reduce a encontrar el valor de $\mu$ para el que

$$P\left(\sum X > 500\right) = P\left(Y > \dfrac{500 - 6\mu}{\sqrt{6}\cdot 15.7}\right)=0.021$$

y por la gráfica, se reduce a resolver la siguiente ecuación:

$$\dfrac{500 - 6\mu}{\sqrt{6}\cdot 15.7} = 2$$

por lo tanto, $\mu$ es igual a:

In [5]:
(500 - np.sqrt(6)*15.7*2)/6

70.51433701276804

**Punto b**

Se van a extraer muestras de tamaño 6, un número de 10 , 100, y 1000 veces. se organizará la info en dataframes y se creará la columna "suma", para determinar con qué frecuencia se obtiene una suma mayor a 500.

In [95]:
lista = []
for i in [10,100,1000]:
    mues = pd.DataFrame(np.random.normal((500 - np.sqrt(6)*15.7*2)/6, 15.7, size=(i, 6)))
    mues['suma']=mues.apply(np.sum,axis=1)
    lista.append((i,(mues[mues['suma']>500].shape[0]/mues.shape[0])*100)) # tupla: (tamaño, Frecuencia relativa porcentual)
pd.DataFrame(lista, columns=['Tamaño', 'Frecuencia (%)'])

,Tamaño,Frecuencia (%)
0,10,0.0
1,100,1.0
2,1000,2.1


## Problema 3

**Amazon ElasticSearch:** El requisito es realizar a "futuro" consultas de los datos de los diferentes accidentes ocurridos, y Elasticsearch es un motor de búsqueda y análisis distribuido, el cual se utiliza habitualmente para análisis de registros, búsqueda de texto completo, inteligencia de seguridad, análisis empresarial y casos de uso de inteligencia operativa.

https://aws.amazon.com/es/opensearch-service/the-elk-stack/what-is-elasticsearch/

## Problema 4

**Amazon RDS:** se requiere un sistema de almacenamiento distribuído, ya que se cuenta con datos de un gran volumen. Amazon Relational Database Service (Amazon RDS) es un servicio web que facilita la configuración, el funcionamiento y la escalabilidad de una base de datos relacional en la nube de AWS, además soporta varios motores de bases de datos para realizar consultas (mySQL, MariaDB, PostgreSQL, entre otros).

https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Welcome.html

## Problema 5

- **episodioId:** Si los episodios están descritos por palabras, se puede usar una nube de palabras para visualizar las palabras más frecuentes.

- **eventoID:** Es realmente un índice. No aplica visualización de distribución, porque cada identificador es único.

- Para analizar las distribuciones de las variables categóricas: **departamento**, **ubicacionOrigen**, **ubicacionDestino**, se pueden usar diagramas de barras (separados).

- **torFscale:** es una variable categórica de escala ordinal, conviene visualizarla con un diagrama de cajas y bigotes.

- **latitudInicial**, **latitudFinal**, **impactoEstimado**: Por la variabilidad de los posibles valores que pueden tomar (dominio), conviene hacer, para cada variable, un historgrama con datos agrupados, definiendo intervalos mediante inrtervalos de clase.

- **muertesTotales**, **lesionesTotales**: Son valores enteros, conviene analizarlos mediante diagramas de caja y bigotes.


## Problema 6

### Punto 1

In [164]:
import imblearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [165]:
cp = pd.read_csv(r'costos_primas.csv')
cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15483 entries, 0 to 15482
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15483 non-null  int64  
 1   state              15483 non-null  object 
 2   group_size         15483 non-null  int64  
 3   homeowner          15483 non-null  int64  
 4   car_age            15483 non-null  int64  
 5   car_value          15435 non-null  object 
 6   risk_factor        15483 non-null  int64  
 7   age_oldest         15308 non-null  float64
 8   age_youngest       15483 non-null  int64  
 9   married_couple     15483 non-null  int64  
 10  C_previous         15483 non-null  int64  
 11  duration_previous  15483 non-null  int64  
 12  A                  15483 non-null  int64  
 13  B                  15483 non-null  int64  
 14  C                  15483 non-null  int64  
 15  D                  15483 non-null  int64  
 16  E                  154

In [166]:
import string
cats = list(string.ascii_uppercase)[:7] # Lista de las primeras 7 letras del alfabeto en mayúsculas
cp[cats] = cp[cats].astype('category') # Columnas de la A a la G como variables categóricas.
cp['car_value'] = cp['car_value'].astype('category')
cp['state'] = cp['state'].astype('category')
cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15483 entries, 0 to 15482
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Unnamed: 0         15483 non-null  int64   
 1   state              15483 non-null  category
 2   group_size         15483 non-null  int64   
 3   homeowner          15483 non-null  int64   
 4   car_age            15483 non-null  int64   
 5   car_value          15435 non-null  category
 6   risk_factor        15483 non-null  int64   
 7   age_oldest         15308 non-null  float64 
 8   age_youngest       15483 non-null  int64   
 9   married_couple     15483 non-null  int64   
 10  C_previous         15483 non-null  int64   
 11  duration_previous  15483 non-null  int64   
 12  A                  15483 non-null  category
 13  B                  15483 non-null  category
 14  C                  15483 non-null  category
 15  D                  15483 non-null  category
 16  E   

Como se puede observar, las únicas variables con datos faltantes son:

    - car_value
    - age_oldest

### Punto 2

De la descripción de las variables dada, se observa que se tienen las siguientes variables categóricas:

    - state
    - homeowner
    - risk_factor
    - married_couple
    - C_previous

In [167]:
cp.head(3) # Imprimir las tres primeras filas para observar sus valores.

,Unnamed: 0,state,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,A,B,C,D,E,F,G,cost
0,0,OK,1,0,9,f,0,24.0,24,0,3,9,0,0,1,1,0,0,4,543
1,1,OK,1,0,9,f,0,24.0,24,0,3,9,2,1,1,3,1,3,2,611
2,2,PA,1,1,7,f,0,74.0,74,0,2,15,2,0,2,3,1,2,2,691


Quitaremos la columna sin nombre, pues trae la misma info que el índice:

In [168]:
cp.drop(cp.iloc[:,[0]],axis=1,inplace=True)
cp.head()

,state,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,A,B,C,D,E,F,G,cost
0,OK,1,0,9,f,0,24.0,24,0,3,9,0,0,1,1,0,0,4,543
1,OK,1,0,9,f,0,24.0,24,0,3,9,2,1,1,3,1,3,2,611
2,PA,1,1,7,f,0,74.0,74,0,2,15,2,0,2,3,1,2,2,691
3,PA,1,1,7,f,0,NaN,74,0,2,15,2,0,2,3,1,2,2,695
4,AR,1,0,4,d,4,26.0,26,0,3,1,1,0,1,1,0,2,2,628


Antes de pasar al siguiente punto, se determinará si los valores de las variables con datos textuales están "normalizados", es decir, que tengan el mismo formato y no ocurra que OK, esté escrito en minúsculas en algunos casos, por ejemplo. Para lo cual, se define una función que imprima los posibles valores de cada variable:

In [169]:
def impr_unic(x,list_car):
    for i in list_car:
        print(f'Posibles valores de la variable {i}: {x[i].unique()}')
        print('--'*50)

In [170]:
impr_unic(cp,['state','car_value'])

Posibles valores de la variable state: ['OK', 'PA', 'AR', 'WA', 'FL', ..., 'WY', 'MT', 'SD', 'ND', 'DC']
Length: 36
Categories (36, object): ['AL', 'AR', 'CO', 'CT', ..., 'WA', 'WI', 'WV', 'WY']
----------------------------------------------------------------------------------------------------
Posibles valores de la variable car_value: ['f', 'd', 'e', 'h', 'g', 'c', 'i', NaN, 'a', 'b']
Categories (9, object): ['a', 'b', 'c', 'd', ..., 'f', 'g', 'h', 'i']
----------------------------------------------------------------------------------------------------


Se observa que los datos están normalizados.

### Punto 3

Conviene realizar el proceso de imputación antes de hacer la codificación, así que se realizarpa el punto 4 primero, y luego se volverpa al punto 3.

### Punto 4

Como se observó antes, las únicas variables con datos faltantes, son: age_oldest, y car_value. La primera es ordinal, la segunda es nominal, así que se imputarán valores faltantes por la moda, para mejorar los resultados, se puede imputar agrupando por otra variable categórica al dataset original como el estado (state), pero en este ejercício sólo se imputará por la moda (por tiempo).

In [172]:
for i in ['car_value','age_oldest']:
    mod = cp[i].mode().iloc[0]
    cp[i] = cp[i].fillna(mod)

In [174]:
cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15483 entries, 0 to 15482
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   state              15483 non-null  category
 1   group_size         15483 non-null  int64   
 2   homeowner          15483 non-null  int64   
 3   car_age            15483 non-null  int64   
 4   car_value          15483 non-null  category
 5   risk_factor        15483 non-null  int64   
 6   age_oldest         15483 non-null  float64 
 7   age_youngest       15483 non-null  int64   
 8   married_couple     15483 non-null  int64   
 9   C_previous         15483 non-null  int64   
 10  duration_previous  15483 non-null  int64   
 11  A                  15483 non-null  category
 12  B                  15483 non-null  category
 13  C                  15483 non-null  category
 14  D                  15483 non-null  category
 15  E                  15483 non-null  category
 16  F   

### Punto 3

One - hot encoding:

In [178]:
X0 = pd.get_dummies(cp, columns = [i for i in cp.columns if i not in ['age_oldest','cost','group_size']])

In [179]:
X0.head()

,group_size,age_oldest,cost,state_AL,state_AR,state_CO,state_CT,state_DC,state_DE,state_FL,...,E_0,E_1,F_0,F_1,F_2,F_3,G_1,G_2,G_3,G_4
0,1,24.0,543,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
1,1,24.0,611,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
2,1,74.0,691,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
3,1,75.0,695,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,1,26.0,628,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0


### Punto 5

In [183]:
from sklearn.model_selection import train_test_split

y = X0['cost']
X= X0.drop(['cost'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=1337)

### Punto 6

In [188]:
import statsmodels.api as sm

model = sm.OLS(endog = Y_train, exog = X_train).fit()

### Punto 7

In [192]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.532
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     71.07
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        16:09:49   Log-Likelihood:                -56862.
No. Observations:               11612   AIC:                         1.141e+05
Df Residuals:                   11428   BIC:                         1.154e+05
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
group_size               0.4537      1.342      0.338      0.735      -2.177       3.084
age_oldest               0.2241      0.051      4.375      0.000       0.124       0.325
state_AL                 0.8491      1.846      0.460      0.645      -2.769       4.467
state_AR                -2.4179      2.495     -0.969      0.333      -7.309       2.473
state_CO                -6.0671      1.738     -3.491      0.000      -9.474      -2.660
state_CT                35.0382      2.133     16.428      0.000      30.858      39.219
state_DC                49.4806      4.529     10.926      0.000      40.603      58.358
state_DE                50.8660      4.104     12.393      0.000      42.821      58.911
state_FL                31.1167      1.294     24.048      0.000      28.580      33.653
state_GA                 9.0247      1.567      5.760      0.000       5.954      12.096
state_IA               -50.3346      2.762    -18.224      0.000     -55.749     -44.921
state_ID                -1.2524      3.660     -0.342      0.732      -8.427       5.922
state_IN                -1.0322      1.788     -0.577      0.564      -4.537       2.472
state_KS                -9.3372      3.889     -2.401      0.016     -16.959      -1.715
state_KY                21.4991      2.083     10.324      0.000      17.417      25.581
state_MD                38.6678      1.685     22.945      0.000      35.364      41.971
state_ME               -31.5621      3.485     -9.058      0.000     -38.392     -24.732
state_MO               -20.7610      2.209     -9.399      0.000     -25.091     -16.431
state_MS                -0.8317      2.627     -0.317      0.752      -5.980       4.317
state_MT               -10.2215      5.008     -2.041      0.041     -20.039      -0.404
state_ND                16.6099      5.595      2.969      0.003       5.643      27.577
state_NE                -8.1654      4.602     -1.774      0.076     -17.186       0.855
state_NH               -18.0024      3.061     -5.882      0.000     -24.002     -12.003
state_NM                 4.5868      3.127      1.467      0.142      -1.543      10.717
state_NV                23.9097      2.063     11.591      0.000      19.866      27.953
state_NY                60.1603      1.644     36.588      0.000      56.937      63.383
state_OH                -8.4592      1.426     -5.934      0.000     -11.254      -5.665
state_OK                -8.5671      1.947     -4.401      0.000     -12.383      -4.751
state_OR                -0.2571      2.047     -0.126      0.900      -4.269       3.755
state_PA                11.9700      1.253      9.555      0.000       9.514      14.426
state_RI                28.0073      3.608      7.764      0.000      20.936      35.079
state_SD               -16.2051     13.153     -1.232    

$R^{2} = 0.532$, $AIC = 1.141\times 10^{5}$ 

Se puede ver que el modelo no presenta un buen ajuste, es necesario realizar una selección de variables, por ejemplo, para mejorar estas métricas.

### Punto 8

El top 5 de los estados con los coeficientes más altos, son aquellos que más valor aportan a la variable de respuesta: cost. De la tabla se puede ver que son, en orden de mayor a menor: NY, DE, DC, MD, y CT.


In [207]:
param = model.params # se puede definir la lista de valores de los parámetros, filtrar porquellas variables que inicien en state. 

### Punto 9

Homeowner es una variable binaria que indica si el cliente tiene casa propia o no, en el nivel 0, el coeficiente del modelo es $80.37$ unidades moinetarias (uu.mm.) lo que indica que cuando el cliente no tiene casa propia, el costo de la prima del seguro aumenta en $80.37$ uu.mm, manteniendo todas las demás características como constantes; en el nivel 1, aumenta en $68.8$ uu.mm. cuando se mantiene todo lo demás constante. En el caso de car_age, se tienen 48 niveles, en el nivel 0, el valor es $16.46$ y se interpreta como un aumento en $16.46$  uu.mm. manteniendo todo lo demás, constante.

### Punto 10

El valor $p$ de cada coeficiente de cada variable (ver tabla arriba) evalúa la hipótesis nula de que el coeficiente es igual a cero (no hay efecto). A un nivel de significancia del $0.01$, un valor $p$ inferior, indica que se puede rechazar la hipótesis nula y en ese caso, la variable correspondiente a ese coeficiente, tendría un efecto en la variable de respuesta (cost). Al ver la tabla se ve que, por mencionar algunas, age_oldest, de state_co a state_IA, de state_KY a state_MO (entre otras) son significativas, así como home_owner_0, home_owner_1, entre muchas otras, como se puede validar, simpelente observando que el valor en la columna $P > |t|$ de la tabla model.summary() sea menor que $0.01$.

## Problema 7

**"Ninguna de las anteriores":** esta sería la respuesta más adecuada, ya que no es posible conocer sistemáticamente el "chance" de que ocurra una u otra situación para poderlos comparar, pues ninguna de las opciones son situaciones intrínsecas del método, sino circunstanciales:

- Que los datos sean "agrupados correctamente", no es el objetivo de este método de segmentación pues se usa como un método descriptivo de aprendizaje no supervisado, en el que se busca determinar, en caso de que existan "estructuras" lo más homogéneas posible internamente y heterogéneas entre sí (minimizar dispersión intra-cluster, o equivalentemente, maximizar la dispersión intra-cluster). 

- El tema de la RAM, depende netamente de ese periférico, no es un limitante del método de segmentación k-means en sí, en caso extremo, puede configurar un cluster de equipos que funcionen en paralelo (pero no es probable que ocurra en un equipo con 32 de RAM si el procesador no es un cuello de botella).

- El desbordamiento numérico es para nada probable, ya que los únicos cálculos que hay que realizar son promedios y distancias, y los rangos numéricos de los datos dados no son grandes, incluso se podría hacer un análisis con un orden de magnitud menor.

- La información obtenida por segmentación con k-means siempre será útil, ya que se busca conocer si existen "estructuras" definidas, o no, en caso de que existan, nos mostrará una idea de esas estructuras y se encontrarán, posiblemnente, los grupos de personas con edades "grandes" que ganan mucho, otro grupo con sueldos "intermedios", sueldos bajos, o los grupos de personas jovenes con sueldos bajos, intermedios, altos, y así sucesivamente.

- El hecho de que un datapoint pertenezca a dos clusters en un momento determinado o cuando converja el algoritmo, depende de los valores particulares que se tengan en el análisis. 